In [ ]:
df.info()

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf 
from statsmodels.tsa.seasonal import seasonal_decompose 
from pmdarima import auto_arima                        
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse
import warnings

In [ ]:
warnings.filterwarnings("ignore")
df = pd.read_csv('/Purple Air Data.csv')
df.head()
df.Date = pd.to_datetime(df.Date)
df = df.set_index("Date")
df.head()

In [ ]:
df_pr = df.copy()
df_pr = df.reset_index()
df_pr.columns = ['ds','y'] # To use prophet column names should be like that
train_data_pr = df_pr.iloc[:len(df)-12]
test_data_pr = df_pr.iloc[len(df)-12:]

In [ ]:
from fbprophet import Prophet
m = Prophet()
m.fit(train_data_pr)
future = m.make_future_dataframe(periods=12,freq='MS')
prophet_pred = m.predict(future)
prophet_pred.tail()

In [ ]:
prophet_pred = pd.DataFrame({"Date" : prophet_pred[-12:]['ds'], "Pred" : prophet_pred[-12:]["yhat"]})
prophet_pred = prophet_pred.set_index("Date")
prophet_pred.index.freq = "MS"
prophet_pred

In [ ]:

test_data["Prophet_Predictions"] = prophet_pred['Pred'].values
import seaborn as sns
plt.figure(figsize=(16,5))
ax = sns.lineplot(x= test_data.index, y=test_data["AQI"])
sns.lineplot(x=test_data.index, y = test_data["Prophet_Predictions"]);

In [ ]:
prophet_rmse_error = rmse(test_data['AQI'], test_data["Prophet_Predictions"])
prophet_mse_error = prophet_rmse_error**2
mean_value = df['AQI'].mean()

print(f'MSE Error: {prophet_mse_error}\nRMSE Error: {prophet_rmse_error}\nMean: {mean_value}')

In [ ]:
# The following code to create a dataframe and remove duplicated rows is always executed and acts as a preamble for your script: 

# dataset = pandas.DataFrame(Date, AQI)
# dataset = dataset.drop_duplicates()

# Paste or type your script code here:
from matplotlib import pyplot as plt
from statsmodels.tsa.arima_model import ARIMA, ARMAResults
from pandas.tseries.offsets import DateOffset
import statsmodels.api as sm
import pandas as pd
import plotly.graph_objects as go
import pandas as pd

dataset = pd.read_csv("Purple Air Data.csv")

dataset['Date']=pd.to_datetime(dataset['Date'])
dataset.set_index('Date',inplace=True)

future_dates=[dataset.index[-1]+ DateOffset(days=x)for x in range(0,24)]


future_datest_dataset = pd.DataFrame(index=future_dates[1:],columns=dataset.columns)

future_dataset=pd.concat([dataset,future_datest_dataset])


model = sm.tsa.statespace.SARIMAX(future_dataset['AQI'].astype(float), order=(2,1,0),seasonal_order=(1,2,0,12))
results = model.fit()
future_dataset['forecast'] = results.predict(start=len(dataset), end= len(dataset)+22, dynamic=True)
future_dataset[['AQI','forecast']].plot()
plt.show()

#future_dataset.to_csv(r'C:\Users\Acer\Desktop\7Mile_future_dataset.csv')